In [8]:
import numpy as np
from sklearn.datasets import fetch_openml 
from sklearn.model_selection import train_test_split

In [4]:
mnist = fetch_openml('mnist_784')

/home/rishabh.mondal/miniconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [7]:
X = mnist.data
y = mnist.target
# print(y)
y = (y.astype(int)==2).astype(int)
# print(y)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)